<a href="https://colab.research.google.com/github/RVS97/MNIST-GANs/blob/master/TCV2_InceptionNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [0]:
import sys
import json
import os
import glob
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.optimizers import Adam, SGD
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose 
from tensorflow.python.keras.layers import Input, UpSampling2D, concatenate  
from tensorflow.python.keras.layers import LeakyReLU
import time
import tensorflow as tf
import numpy as np
import cv2
import random
from sklearn.utils import shuffle
from matplotlib import pyplot
from math import ceil
from scipy.stats import describe
from sklearn.model_selection import train_test_split
import time
import pickle
import datetime

# Dataset
Import MNIST dataset from keras and map image values to 0/1

In [0]:
# Import dataset
from keras.datasets import mnist

class MNISTdata:
  def __init__(self, batchSize=100, shuffle=True, vBatchNorm=False):
    
    # Load into variables
    (self.x_train, self.y_train),(self.x_test, self.y_test) = mnist.load_data()

    self.x_train = np.expand_dims(self.x_train, -1)
    self.x_test = np.expand_dims(self.x_test, -1)
    
    # One-hot encode labels
    self.y_train = to_categorical(self.y_train, num_classes=10)
    self.y_test = to_categorical(self.y_test, num_classes=10)
    
    # Map image values to the range -1/1
    self.x_train = (self.x_train.astype(np.float32) - 127.5)/127.5*-1
    self.x_test = (self.x_test.astype(np.float32) - 127.5)/127.5*-1

    self.imgWidth = len(self.x_train[0][0])
    self.imgHeight = len(self.x_train[0])
    self.nTrainSamples = len(self.x_train)
    self.nTestSamples = len(self.x_test)
    
    print("MNIST loaded correctly")
    print(" - {} by {} images (grayscale)".format(self.imgWidth,self.imgHeight))
    print(" - {} training samples".format(self.nTrainSamples))
    print(" - {} test samples".format(self.nTestSamples))
    
    self.batchSize = batchSize
    self.nBatches = ceil(self.nTrainSamples/self.batchSize)
    
    print("Batch size {} -> {} batches".format(self.batchSize, self.nBatches))
    
    if shuffle: self.shuffleData()
    
  def shuffleData(self):
    self.x_train, self.y_train = shuffle(self.x_train, self.y_train)
    
  def getBatch(self, batchId):
    if (batchId+1)*self.batchSize > self.nTrainSamples:
      batch = self.x_train[batchId*self.batchSize:]
    else:
      batch = self.x_train[batchId*self.batchSize:(batchId+1)*self.batchSize]
    return batch, np.ones((len(batch),))    
    
data = MNISTdata(batchSize = 200)

MNIST loaded correctly
 - 28 by 28 images (grayscale)
 - 60000 training samples
 - 10000 test samples
Batch size 200 -> 300 batches


# Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
rootDir = "/content/rootDir"

def saveFile(filePath, rootDir):
  !cp $filePath $rootDir
  
def getFile(fileName, rootDir, localDir = "./"):
  path = rootDir + "/" + fileName
  !cp $path $localDir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Classification Network

In [0]:
# Simple Network is enough

def getClassifier(inputShape=(28,28,1)):
  model = Sequential(name="Discriminator")
  model.add(Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same', input_shape=inputShape, kernel_initializer='glorot_uniform'))
  model.add(BatchNormalization(axis=-1))

  model.add(Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same', kernel_initializer='glorot_uniform'))
  model.add(BatchNormalization(axis=-1))
  
  model.add(Flatten())
  model.add(Dense(50, activation='relu', kernel_initializer='glorot_uniform'))
  model.add(BatchNormalization(axis=-1))

  model.add(Dense(10, activation='sigmoid', kernel_initializer='glorot_uniform'))
  return model

# Train Network

In [0]:
classNet = getClassifier()
classNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
hist = classNet.fit(data.x_train, data.y_train, batch_size=200, epochs=15)

In [0]:
pyplot.plot(hist.history['loss'])

#Evaluate Accuracy

In [0]:
classNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
testEval = classNet.evaluate(data.x_test, data.y_test)